In [1]:
from data import TargetData

d = TargetData()


In [2]:
d.train.sample(10)

,uid,mid,rating
764867,4555,620,1
379365,2218,26,1
785872,4702,873,1
669771,4027,1336,1
364645,2123,2756,1
533913,3298,184,1
720345,4312,2077,1
405020,2426,620,1
246144,1488,1594,1
5522,38,184,1


In [3]:
d.df

,uid_old,mid_old,rating,date,uid,mid,latest
0,1,1193,1,2000-12-31 22:12:40,0,0,42.0
1,1,661,1,2000-12-31 22:35:09,0,1,23.0
2,1,914,1,2000-12-31 22:32:48,0,2,28.0
3,1,3408,1,2000-12-31 22:04:35,0,3,47.0
4,1,2355,1,2001-01-06 23:38:11,0,4,4.0
...,...,...,...,...,...,...,...
999606,6040,1091,1,2000-04-26 02:35:41,6039,772,161.0
999607,6040,1094,1,2000-04-25 23:21:27,6039,1106,293.0
999608,6040,562,1,2000-04-25 23:19:06,6039,365,305.0
999609,6040,1096,1,2000-04-26 02:20:48,6039,152,234.0


In [4]:
from random import choice
# movies grouped by user
def all_movies_by_usr(train, test):
    movies_all = zip(train.groupby('uid'), test.groupby('uid'))
    return [(set(tr[1].mid.tolist()), set(te[1].mid.tolist())) if tr[1].uid.iloc[0] == te[1].uid.iloc[0] else '' for tr, te in movies_all]

# movies in train and movies in test for each is user
# user0 = None
def check_for_leaks(train, test):
    # x = 0
    leaks = 0
    u = choice(range(test.uid.nunique()))
    for i, (tr, te) in enumerate( all_movies_by_usr(train, test)):
        if i == u:
            # tr.add(25)  # add test movie to train
            print(f"User: {u} watched these movies:")
            print(f"movies in train {sorted(list(tr)[:5])}")
            print(f"\tmovies in test {sorted(list(te)[:5])}")
            if tr.intersection(te):
                print(f'user: {i} has {tr.intersection(te)} in both train/test')
                leaks += 1
    if leaks > 1:
        print(f"*** {leaks} Leaks ***")
    elif leaks == 1:
        print(f"*** {leaks} Leak ***")
    else:
        print('*** 0 Leaks Found ***')

u = check_for_leaks(d.train, d.test)

User: 5358 watched these movies:
movies in train [0, 1, 3, 4, 5]
	movies in test [136]
*** 0 Leaks Found ***


In [5]:
from torch import LongTensor
from data import AttributeData
features = AttributeData()

/Users/franciscocancedda/Documents/Scool/NFPCF/data.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['rating'] = 1
/Users/franciscocancedda/Documents/Scool/NFPCF/data.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['uid'] = clean.uid - 1


In [6]:
features.df

,uid,gender,age,ojob,rating,job
0,0,1,56,16,1,0
1,1,1,25,15,1,1
2,2,1,45,7,1,2
3,3,1,25,20,1,3
4,4,0,50,9,1,4
...,...,...,...,...,...,...
4915,6033,0,25,1,1,5
4916,6034,0,25,15,1,1
4917,6035,0,45,1,1,5
4918,6036,0,56,1,1,5


In [7]:
import pandas as pd

df = pd.merge(d.df, features.df, on=['uid', 'uid'], how='left')

df.uid.nunique()


6040

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999611 entries, 0 to 999610
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   uid_old   999611 non-null  int64         
 1   mid_old   999611 non-null  int64         
 2   rating_x  999611 non-null  int8          
 3   date      999611 non-null  datetime64[ns]
 4   uid       999611 non-null  int64         
 5   mid       999611 non-null  int64         
 6   latest    999611 non-null  float64       
 7   gender    828114 non-null  float64       
 8   age       828114 non-null  float64       
 9   ojob      828114 non-null  float64       
 10  rating_y  828114 non-null  float64       
 11  job       828114 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4), int8(1)
memory usage: 92.5 MB


In [9]:
fd = pd.merge(d.df,features.df, on=['uid', 'uid'], how='right')
fd.uid.nunique()

4920

In [10]:
import torch
import numpy as np
from random import sample
from time import time

class LoadData(torch.utils.data.Dataset):
    def __init__(self):
        t1 = time()
        self.ratings = self._load_ratings()
        t2 = time()
        self.user_features = self._load_features()
        t3 = time()
        self.fd = pd.merge(self.ratings,self.user_features, on=['uid', 'uid'], how='right')
        t4 = time()
        self.fd = self._reset_index(self.fd, ['uid', 'mid', 'job'])
        print(self.fd.head())
        self.train, self.test = self._train_test_split()
        movies = set(self.fd.mid.unique())
        self.tr = self.add_negatives(self.train, items = movies, n_samples=5)
        self.te = self.add_negatives(self.test, items= movies, tr=self.train, n_samples=100)
        self.testing_tensors = self.parse_testing(self.te)

        t6 = time()
        print('ratings', t2 - t1)
        print('features', t3 - t2)
        print('merge', t4 - t3)
        print('negatives', t6 - t4)
        print('total',t6-t1)

    def __len__(self):
        return self.tr.shape[0]

    def __getitem__(self, item):
        u, m, r = self.tr[['uid', 'mid', 'rating']].iloc[item]
        return LongTensor([u]), LongTensor([m]), LongTensor([r])

    @staticmethod
    def add_negatives(df: pd.DataFrame, item: str = 'mid', items=None, tr = None, n_samples: int = 4):
        if items is None:
            items = set(df[item].unique())

        def user_movies(uid, mid):
            train_movies = set()
            if tr is not None:
                train_movies = set(tr[tr.uid==uid].mid.unique())
                if not train_movies:
                    print(' no movies in train found', uid)
            return sample(list(items - train_movies - mid), n_samples)

        df['rating'] = np.int8(1)
        combine = df.groupby('uid')[item].apply(set).reset_index()
        combine['negatives'] = combine.apply(lambda x: user_movies(x.uid ,x.mid), axis=1)

        s = combine.apply(lambda x: pd.Series(x.negatives, dtype=np.int16), axis=1).stack().reset_index()
        s.rename(columns={'level_0': 'uid', 0: item}, inplace=True)
        s.drop(['level_1'], axis=1, inplace=True)
        s['rating'] = np.int8(0)
        s.uid = s.uid.astype(np.int16)

        # complete = pd.merge([df, s], how='left', on='uid').sort_values(by=['uid', item])
        complete = pd.concat([df, s]).sort_values(by=['uid', item])
        return complete.reset_index(drop=True)

    def _train_test_split(self):
        self.fd['latest'] = self.fd.groupby(['uid'])['date'].rank(method='first', ascending=False)
        test_bool = self.fd.latest <= 1
        test = self.fd[test_bool]
        train = self.fd[~test_bool]
        return (
                    train[['uid', 'mid']], test[['uid', 'mid']]
            )

    @staticmethod
    def _reset_index(df, cols):
        for col in cols:
            old_col = col + '_old'
            df.rename(columns={col: old_col}, inplace=True)
            user_id = df[[old_col]].drop_duplicates().reindex()
            user_id[col] = np.arange(len(user_id))
            df = pd.merge(df, user_id, on=[old_col], how='left')
        return df


    @staticmethod
    def _load_ratings(min_ratings=5):
        df = pd.read_csv('MovieLens/ratings.dat',
                         sep='::',
                         header=None,
                         names=['uid', 'mid', 'rating', 'date'],
                         parse_dates=['date'],
                         date_parser=lambda x: pd.to_datetime(x, unit='s', origin='unix'),
                         engine='python')

        # DROP MOVIES WITH LESS THAN 5 RATINGS
        s = df.groupby(['mid']).size()
        low_n_ratings = s[s < min_ratings].reset_index().mid.tolist()
        return df[~df.mid.isin(low_n_ratings)]


    @staticmethod
    def _load_features():
        df = pd.read_csv('MovieLens/users.dat',
                         sep='::',
                         header=None,
                         names=['uid', 'gender', 'age', 'job', 'zip'],
                         engine='python')
        df.gender = pd.get_dummies(df.gender, drop_first=True)  # 0:F, 1:M
        drop = [0, 10, 13, 19]

        clean = df[~df['job'].isin(drop)]
        return clean.drop(columns=['zip'])

    @staticmethod
    def parse_testing(df):
        test = df.sort_values(by=['uid', 'rating'], ascending=False)
        users, movies, outputs = [], [], []
        for _, u in test.groupby('uid'):
            users.append(LongTensor(u.uid.to_numpy()))
            # users.append(LongTensor([u.uid.values]))
            movies.append(LongTensor(u.mid.to_numpy()))
            # movies.append(LongTensor([u.mid.values]))
            outputs.append(LongTensor(u.rating.to_numpy()))
            # outputs.append(LongTensor([u.rating.values]))
        return users, movies, outputs


data = LoadData()

   uid_old  mid_old  rating                date  gender  age  job_old  uid  \
0        2     1357       5 2000-12-31 21:38:29       1   56       16    0   
1        2     3068       4 2000-12-31 21:43:20       1   56       16    0   
2        2     1537       4 2000-12-31 21:53:40       1   56       16    0   
3        2      647       3 2000-12-31 21:49:11       1   56       16    0   
4        2     2194       4 2000-12-31 21:48:17       1   56       16    0   

   mid  job  
0    0    0  
1    1    0  
2    2    0  
3    3    0  
4    4    0  
ratings 8.178212881088257
features 0.030310869216918945
merge 0.13698911666870117
negatives 9.804809093475342
total 18.15032196044922


In [11]:
check_for_leaks(data.tr, data.te)


User: 2157 watched these movies:
movies in train [11, 16, 257, 904, 2306]
	movies in test [1030, 1033, 1543, 3079, 3080]
*** 0 Leaks Found ***


In [12]:
num_epochs = 200
batch_size = 128
learning_rate = .05

emb_size = 128  # LATENT DIM

hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1

random_samples = 100
num_negatives = 4
top_k = 10

In [13]:
from models import NCF
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_users = data.tr.uid.nunique()
n_movies = data.tr.mid.nunique()
ncf = NCF(n_users, n_movies, emb_size, hidden_layers, output_size).to(device)


In [ ]:
from evaluators import eval_model
optimizer = torch.optim.SGD(ncf.parameters(), lr=learning_rate, weight_decay=1e-6)

dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=0)
it_per_epoch = len(data) / batch_size

loss = 0
for i in range(num_epochs):
    ncf.train()
    print("Starting epoch ", i + 1)
    j = 0
    t1 = time()
    for batch in dataloader:
        u, m, r = batch
        # move tensors to cuda
        u = u.to(device)
        m = m.to(device)
        r = r.to(device)

        y_hat = ncf(u.squeeze(1), m.squeeze(1))
        loss = torch.nn.BCELoss()  # (weight=w, reduction="mean")

        loss = loss(y_hat, r.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if j % int(1 + it_per_epoch / 10) == 0:
            print(f"\rProgress: {round(100 * j / it_per_epoch)}%", end='')
        j+=1

    t2 = time()
    print("Epoch time:", round(t2 - t1), "seconds")
    print("Loss:", loss.cpu().detach().numpy().round(3))
    ncf.eval()
    hr, ndcg = eval_model(ncf, data, n_users, device)

    print(f"HR@{top_k}: {round(hr, 2)}  NDCG@{top_k}: {round(ndcg, 2)}\n")
print("Done")

Starting epoch  1
Progress: 70%